In [8]:
import numpy as np

In [2]:
def sigmoidal(x):
    z = 1/(1 + np.exp(-x))
    return(z)

def bipolarSigmoidal(x):
    z = np.exp(-x)
    z = (1-z)/(1+z)
    return(z)

In [6]:
def actiVal(x):
    z = sigmoidal(x)
    return(z*(1-z))

def actiValBS(x):
    z = bipolarSigmoidal(x)
    return(1*(1+z)*(1-z))

def getBothBS(x):
    print(bipolarSigmoidal(x))
    print(actiValBS(x))

def getBoth(x):
    print(sigmoidal(x))
    print(actiVal(x))

In [50]:
class BackPropagation():
    def __init__(self, lr, n, p, m):
        self.lr = lr
        self.n = n
        self.p = p
        self.m = m
        self.x = np.zeros((n,))
        self.v = np.zeros((n+1, p))
        self.deltain = np.zeros((p,))
        self.deltaopZ = np.zeros((p,))
        self.w = np.zeros((p+1, m))
        self.deltaw = np.zeros((p+1, m))
        self.zin = np.zeros((p,))
        self.zout = np.zeros((p,))
        self.yin = np.zeros((m,))
        self.yout = np.zeros((m,))
        self.deltaop = np.zeros((m,))
        self.deltav = np.zeros((n+1, p))

    def initInputs(self, x, v, w):
        for i in range(self.n):
            self.x[i] = x[i]
        for i in range(self.n+1):
            for j in range(self.p):
                self.v[i][j] = v[i][j]
        for i in range(self.p+1):
            for j in range(self.m):
                self.w[i][j] = w[i][j]
    
    def forwardPropagationZ(self, actfn):
        for i in range(self.p):
            for j in range(self.n):
                self.zin[i] += self.v[j+1][i]*self.x[j]
                
            self.zin[i] += self.v[0][i]
            self.zout[i] = actfn(self.zin[i])
            print("zin_{} = {}".format(i+1, self.zin[i]))
            print("zout_{} = {}".format(i+1, self.zout[i]))
    
    def forwardPropagationY(self, actfn):
        for i in range(self.m):
            for j in range(self.p):
                self.yin[i] += self.w[j+1][i]*self.zout[j]
            self.yin[i] += self.w[0][i]
            self.yout[i] = actfn(self.yin[i])
            print("yin_{} = {}".format(i+1, self.yin[i]))
            print("yout_{} = {}".format(i+1, self.yout[i]))
    
    def backwardPropagationY(self, actfndiff, t):
        for i in range(self.m):
            self.deltaop[i] = (t-self.yout[i])*(actfndiff(self.yin[i]))
            print("Error: {}".format(t-self.yout[i]))
            print("Delta_{} = {}".format(i, self.deltaop[i]))
        for i in range(self.m):
            for j in range(self.p):
                self.deltaw[j+1][i] = self.lr * self.deltaop[i] * self.zout[j]
                print("deltaw_{}{} = {}".format(j+1, i, self.deltaw[j+1][i]))
            self.deltaw[0][i] = self.lr * self.deltaop[i]
            print("deltaw_{}{} = {}".format(0, i, self.deltaw[0][i]))
    
    def backwardPropagationZ(self, actfndiff):
        for i in range(self.p):
            for j in range(self.m):
                self.deltain[i] += self.deltaop[j] * self.w[i+1][j]
            print("deltain_{} = {}".format(i, self.deltain[i]))
        for i in range(self.p):
            self.deltaopZ[i] = self.deltain[i] * actfndiff(self.zin[i])
        for i in range(self.p):
            for j in range(self.n):
                self.deltav[j+1][i] = self.lr * self.deltaopZ[i] * self.x[j]
                print("deltv_{}{} = {}".format(j+1, i, self.deltav[j+1][i]))
            self.deltav[0][i] = self.lr * self.deltaopZ[i]
            print("deltav_{}{} = {}".format(0, i, self.deltav[0][i]))

In [51]:
bp = BackPropagation(0.25, 2, 2, 1)

In [52]:
bp.initInputs(np.array([0, 1]), np.array([[0.3, 0.5], [0.6, -0.3], [-0.1, 0.4]]), np.array([[-0.2], [0.4], [0.1]]))

In [53]:
bp.forwardPropagationZ(sigmoidal)

zin_1 = 0.19999999999999998
zout_1 = 0.549833997312478
zin_2 = 0.9
zout_2 = 0.7109495026250039


In [54]:
bp.forwardPropagationY(sigmoidal)

yin_1 = 0.09102854918749159
yout_1 = 0.5227414361305817


In [55]:
bp.backwardPropagationY(actiVal, t = np.array([1]))

Error: [0.47725856]
Delta_0 = 0.11906781576358075
deltaw_10 = 0.01636688327313882
deltaw_20 = 0.021162801098940833
deltaw_00 = 0.029766953940895187


In [56]:
bp.backwardPropagationZ(actiVal)

deltain_0 = 0.0476271263054323
deltain_1 = 0.011906781576358076
deltv_10 = 0.0
deltv_20 = 0.0029471257678088682
deltav_00 = 0.0029471257678088682
deltv_11 = 0.0
deltv_21 = 0.0006117118183496963
deltav_01 = 0.0006117118183496963
